# Imports

In [1]:
# FYI:
# This pip command takes a lot with GPU enabled (~15 min)
# It works though. And GPU accelerates the process *a lot*.
# I am developing with GPU turned off and submitting with GPU turned on
!pip install --upgrade implicit

You should consider upgrading via the 'C:\Users\mdurh\anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
import os; os.environ['OPENBLAS_NUM_THREADS']='1'
import sys

import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

from sklearn.preprocessing import MinMaxScaler

import numpy as np
import pandas as pd
import implicit
from scipy.sparse import coo_matrix
from implicit.evaluation import mean_average_precision_at_k

# Load dataframes

In [ ]:
#transaction data 
#users 
#items 

In [127]:
df = pd.read_csv('cleaned_data2.csv', dtype={'StockCode': str}, parse_dates=['InvoiceDate'])

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 516528 entries, 0 to 516527
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    516528 non-null  object        
 1   StockCode    516528 non-null  object        
 2   Quantity     516528 non-null  int64         
 3   InvoiceDate  516528 non-null  datetime64[ns]
 4   UnitPrice    516528 non-null  float64       
 5   CustomerID   392980 non-null  float64       
 6   Country      516528 non-null  object        
 7   TotalValue   516528 non-null  float64       
 8   Description  516528 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 35.5+ MB


In [129]:
df.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalValue,Description
0,536365,85123a,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,white hanging heart t-light holder
1,536365,71053,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,white metal lantern
2,536365,84406b,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,cream cupid hearts coat hanger
3,536365,84029g,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,knitted union flag hot water bottle
4,536365,84029e,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,red woolly hottie white heart.


In [130]:
df_cust = df.dropna()
df_cust = df_cust[df_cust["Quantity"]>0]
dfu=pd.DataFrame(df_cust.groupby(["CustomerID","StockCode"],as_index=False)["Quantity"].sum())
dfu.head()

,CustomerID,StockCode,Quantity
0,12346.0,23166,74215
1,12347.0,16008,24
2,12347.0,17021,36
3,12347.0,20665,6
4,12347.0,20719,40


In [131]:
#sanity check
df_cust[(df_cust["CustomerID"]==12347.0)&(df_cust["StockCode"]=="20719")]

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalValue,Description
67113,542237,20719,10,2011-01-26 14:30:00,0.85,12347.0,Iceland,8.5,woodland charlotte bag
270288,562032,20719,10,2011-08-02 08:48:00,0.85,12347.0,Iceland,8.5,woodland charlotte bag
407908,573511,20719,10,2011-10-31 12:25:00,0.85,12347.0,Iceland,8.5,woodland charlotte bag
509978,581180,20719,10,2011-12-07 15:52:00,0.85,12347.0,Iceland,8.5,woodland charlotte bag


In [102]:
# For validation this means 3 weeks of training and 1 week for validation
# For submission, it means 4 weeks of training
df['InvoiceDate'].max()

Timestamp('2011-12-09 12:50:00')

## Assign autoincrementing ids starting from 0 to both users and items

In [132]:
# Create a numeric user_id and item_id column
dfu['CustomerID'] = dfu['CustomerID'].astype("category")
dfu['StockCode'] = dfu['StockCode'].astype("category")
dfu['user_id'] = dfu['CustomerID'].cat.codes
dfu['item_id'] = dfu['StockCode'].cat.codes

In [142]:
dfu["item_id"].max()

2486

In [134]:
# The implicit library expects data as a item-user matrix so we
# create two matricies, one for fitting the model (item-user) 
# and one for recommendations (user-item)
sparse_item_user = sparse.csr_matrix((dfu['Quantity'].astype(float), (dfu['item_id'], dfu['user_id'])))
sparse_user_item = sparse.csr_matrix((dfu['Quantity'].astype(float), (dfu['user_id'], dfu['item_id'])))

In [135]:
#used for refering back to original ids later
customer_reference = dfu[["CustomerID",'user_id']].drop_duplicates()
item_reference = dfu[["StockCode",'item_id']].drop_duplicates()
item_reference=pd.merge(item_reference,df_cust[["StockCode","Description"]], on="StockCode", how="left").drop_duplicates()
customer_reference[customer_reference["CustomerID"]==12350.0]

,CustomerID,user_id
192,12350.0,4


In [145]:
# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=30, regularization=0.1, iterations=20)

# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 10
data_conf = (sparse_item_user * alpha_val).astype('double')

#Fit the model
model.fit(data_conf)

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
#evaluate

In [146]:
model.similar_items(4289, 12)

(array([4289, 4026, 3103,  918, 2021,    4,  813, 1954, 1173, 1237, 1822,
         660]),
 array([1.0000001 , 0.65238154, 0.6232579 , 0.56579095, 0.561326  ,
        0.56018925, 0.55306983, 0.5495835 , 0.53964865, 0.53734595,
        0.52323115, 0.51966226], dtype=float32))

In [139]:
#---------------------
# FIND SIMILAR ITEMS
#---------------------

item_id = 10 #
n_similar = 10

# Use implicit to get similar items.
similar = model.similar_items(item_id, n_similar)

# Print the names of our most similar items
for item in similar[0]:
#    print(dfu.StockCode[dfu.item_id == item].unique())
    print(similar)


(array([  10, 3105, 1345, 1278, 1290, 3535, 2299, 2637, 2355, 2470]), array([0.99999994, 0.78041   , 0.71575505, 0.6970002 , 0.67876244,
       0.67435926, 0.671706  , 0.6301514 , 0.62654394, 0.62219477],
      dtype=float32))
(array([  10, 3105, 1345, 1278, 1290, 3535, 2299, 2637, 2355, 2470]), array([0.99999994, 0.78041   , 0.71575505, 0.6970002 , 0.67876244,
       0.67435926, 0.671706  , 0.6301514 , 0.62654394, 0.62219477],
      dtype=float32))
(array([  10, 3105, 1345, 1278, 1290, 3535, 2299, 2637, 2355, 2470]), array([0.99999994, 0.78041   , 0.71575505, 0.6970002 , 0.67876244,
       0.67435926, 0.671706  , 0.6301514 , 0.62654394, 0.62219477],
      dtype=float32))
(array([  10, 3105, 1345, 1278, 1290, 3535, 2299, 2637, 2355, 2470]), array([0.99999994, 0.78041   , 0.71575505, 0.6970002 , 0.67876244,
       0.67435926, 0.671706  , 0.6301514 , 0.62654394, 0.62219477],
      dtype=float32))
(array([  10, 3105, 1345, 1278, 1290, 3535, 2299, 2637, 2355, 2470]), array([0.99999994, 0.7

In [54]:
model.similar_items(item_id, n_similar)

(array([  50, 3665, 1083,  215, 2033, 3081, 4263,  293,  466, 4044]),
 array([1.        , 0.78307253, 0.76667935, 0.7500872 , 0.7462784 ,
        0.72889173, 0.7096694 , 0.6820137 , 0.6792386 , 0.6709201 ],
       dtype=float32))

In [98]:
user_id = 28

# Use the implicit recommender.
recommended = model.recommend(user_id, sparse_user_item[user_id])

recomendations_list = recommended[0].tolist()
items = []
description = []
score = []


# Get artist names from ids
for item in recomendations_list:
    items.append(dfu["StockCode"][dfu.item_id == item].unique().tolist())

recommendations = pd.DataFrame({'items': items, 'score': recommended[1]})
print(recommendations)

      items     score
0   [20685]  1.158388
1        []  1.150790
2        []  1.147403
3  [84968a]  1.145483
4   [21366]  1.145337
5   [23281]  1.135852
6        []  1.133452
7  [84912a]  1.123729
8        []  1.117575
9   [21832]  1.115110


In [84]:
recomendations_list = recommended[0].tolist()
recomendations_list

[66, 3695, 3911, 2323, 326, 1757, 3823, 2300, 4002, 558]

In [83]:
dfu.StockCode[dfu.item_id.isin(recomendations_list)].unique()

['21832', '20685', '84912a', '23281', '84968a', '21366']
Categories (6, object): ['21832', '20685', '84912a', '23281', '84968a', '21366']

In [147]:
dfu[dfu.item_id == 3695]

,CustomerID,StockCode,Quantity,user_id,item_id


## Create coo_matrix (user x item) and csr matrix (user x item)

It is common to use scipy sparse matrices in recommender systems, because the main core of the problem is typically modeled as a matrix with users and items, with the values representing whether the user purchased (or liked) an items. Since each user purchases only a small fraction of the catalog of products, this matrix is full of zero (aka: it's sparse).

In a very recent release they did an API breaking change, so be aware of that: https://github.com/benfred/implicit/releases
In this notebook we are using the latest version, so everything is aligned with (user x item)

**We are using (user x item) matrices, both for training and for evaluating/recommender.**

In the previous versions the training procedure required a COO item x user

For evaluation and prediction, on the other hand, CSR matrices with users x items format should be provided.


### About COO matrices
COO matrices are a kind of sparse matrix.
They store their values as tuples of `(row, column, value)` (the coordinates)

You can read more about them here: 
* https://en.wikipedia.org/wiki/Sparse_matrix#Coordinate_list_(COO)
* https://scipy-lectures.org/advanced/scipy_sparse/coo_matrix.html

From https://het.as.utexas.edu/HET/Software/Scipy/generated/scipy.sparse.coo_matrix.html

```python
>>> row  = np.array([0,3,1,0]) # user_ids
>>> col  = np.array([0,3,1,2]) # item_ids
>>> data = np.array([4,5,7,9]) # a bunch of ones of lenght unique(user) x unique(items)
>>> coo_matrix((data,(row,col)), shape=(4,4)).todense()
matrix([[4, 0, 9, 0],
        [0, 7, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 5]])
```

## About CSR matrices
* https://en.wikipedia.org/wiki/Sparse_matrix#Compressed_sparse_row_(CSR,_CRS_or_Yale_format)


In [27]:
row = df['user_id'].values
col = df['item_id'].values
data = np.ones(df.shape[0])
coo_train = coo_matrix((data, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))
coo_train

<4356x2489 sparse matrix of type '<class 'numpy.float64'>'
	with 516528 stored elements in COOrdinate format>

# Check that model works ok with data

In [64]:
%%time
model = implicit.als.AlternatingLeastSquares(factors=10, iterations=2)
model.fit(data_conf)

  0%|          | 0/2 [00:00<?, ?it/s]

Wall time: 415 ms


# Validation

## Functions required for validation

In [65]:
def to_user_item_coo(df):
    """ Turn a dataframe with transactions into a COO sparse items x users matrix"""
    row = df['user_id'].values
    col = df['item_id'].values
    data = np.ones(df.shape[0])
    coo = coo_matrix((data, (row, col)), shape=(len(ALL_USERS), len(ALL_ITEMS)))
    return coo


def split_data(df, validation_days=7):
    """ Split a pandas dataframe into training and validation data, using <<validation_days>>
    """
    validation_cut = df['InvoiceDate'].max() - pd.Timedelta(validation_days)

    df_train = df[df['InvoiceDate'] < validation_cut]
    df_val = df[df['InvoiceDate'] >= validation_cut]
    return df_train, df_val

def get_val_matrices(df, validation_days=7):
    """ Split into training and validation and create various matrices
        
        Returns a dictionary with the following keys:
            coo_train: training data in COO sparse format and as (users x items)
            csr_train: training data in CSR sparse format and as (users x items)
            csr_val:  validation data in CSR sparse format and as (users x items)
    
    """
    df_train, df_val = split_data(df, validation_days=validation_days)
    coo_train = to_user_item_coo(df_train)
    coo_val = to_user_item_coo(df_val)

    csr_train = coo_train.tocsr()
    csr_val = coo_val.tocsr()
    
    return {'coo_train': coo_train,
            'csr_train': csr_train,
            'csr_val': csr_val
          }


def validate(matrices, factors=200, iterations=20, regularization=0.01, show_progress=True):
    """ Train an ALS model with <<factors>> (embeddings dimension) 
    for <<iterations>> over matrices and validate with MAP@12
    """
    coo_train, csr_train, csr_val = matrices['coo_train'], matrices['csr_train'], matrices['csr_val']
    
    model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                 iterations=iterations, 
                                                 regularization=regularization, 
                                                 random_state=42)
    model.fit(coo_train, show_progress=show_progress)
    
    # The MAPK by implicit doesn't allow to calculate allowing repeated items, which is the case.
    # TODO: change MAP@12 to a library that allows repeated items in prediction
    map12 = mean_average_precision_at_k(model, csr_train, csr_val, K=12, show_progress=show_progress, num_threads=4)
    print(f"Factors: {factors:>3} - Iterations: {iterations:>2} - Regularization: {regularization:4.3f} ==> MAP@12: {map12:6.5f}")
    return map12

In [66]:
matrices = get_val_matrices(data_conf)

IndexError: Index dimension must be <= 2

In [35]:
%%time
best_map12 = 0
for factors in [40, 50, 60, 100, 200, 500, 1000]:
    for iterations in [3, 12, 14, 15, 20]:
        for regularization in [0.01]:
            map12 = validate(matrices, factors, iterations, regularization, show_progress=False)
            if map12 > best_map12:
                best_map12 = map12
                best_params = {'factors': factors, 'iterations': iterations, 'regularization': regularization}
                print(f"Best MAP@12 found. Updating: {best_params}")

Factors:  40 - Iterations:  3 - Regularization: 0.010 ==> MAP@12: 0.00758
Best MAP@12 found. Updating: {'factors': 40, 'iterations': 3, 'regularization': 0.01}
Factors:  40 - Iterations: 12 - Regularization: 0.010 ==> MAP@12: 0.00694
Factors:  40 - Iterations: 14 - Regularization: 0.010 ==> MAP@12: 0.00694
Factors:  40 - Iterations: 15 - Regularization: 0.010 ==> MAP@12: 0.00000
Factors:  40 - Iterations: 20 - Regularization: 0.010 ==> MAP@12: 0.00000
Factors:  50 - Iterations:  3 - Regularization: 0.010 ==> MAP@12: 0.01190
Best MAP@12 found. Updating: {'factors': 50, 'iterations': 3, 'regularization': 0.01}
Factors:  50 - Iterations: 12 - Regularization: 0.010 ==> MAP@12: 0.00926
Factors:  50 - Iterations: 14 - Regularization: 0.010 ==> MAP@12: 0.00926
Factors:  50 - Iterations: 15 - Regularization: 0.010 ==> MAP@12: 0.00926
Factors:  50 - Iterations: 20 - Regularization: 0.010 ==> MAP@12: 0.01042
Factors:  60 - Iterations:  3 - Regularization: 0.010 ==> MAP@12: 0.01042
Factors:  60 -

In [36]:
del matrices

# Training over the full dataset

In [37]:
coo_train = to_user_item_coo(df)
csr_train = coo_train.tocsr()

In [38]:
def train(coo_train, factors=200, iterations=15, regularization=0.01, show_progress=True):
    model = implicit.als.AlternatingLeastSquares(factors=factors, 
                                                 iterations=iterations, 
                                                 regularization=regularization, 
                                                 random_state=42)
    model.fit(coo_train, show_progress=show_progress)
    return model

In [39]:
best_params

{'factors': 500, 'iterations': 20, 'regularization': 0.01}

In [40]:
model = train(coo_train, **best_params)

  0%|          | 0/20 [00:00<?, ?it/s]

# Submission

## Submission function

In [41]:
def submit(model, csr_train, submission_name="submissions.csv"):
    preds = []
    batch_size = 2000
    to_generate = np.arange(len(ALL_USERS))
    for startidx in range(0, len(to_generate), batch_size):
        batch = to_generate[startidx : startidx + batch_size]
        ids, scores = model.recommend(batch, csr_train[batch], N=12, filter_already_liked_items=False)
        for i, userid in enumerate(batch):
            customer_id = user_ids[userid]
            user_items = ids[i]
            article_ids = [item_ids[item_id] for item_id in user_items]
            preds.append((customer_id, ' '.join(article_ids)))

    df_preds = pd.DataFrame(preds, columns=['customer_id', 'prediction'])
    df_preds.to_csv(submission_name, index=False)
    
    display(df_preds.head())
    print(df_preds.shape)
    
    return df_preds

In [42]:
%%time
df_preds = submit(model, csr_train);

,customer_id,prediction
0,17850.0,71477 82494l 82483 82482 84029g 85123a 84029e ...
1,13047.0,22961 22090 22692 22077 22969 21755 23245 2262...
2,12583.0,22417 22551 22617 21094 22556 22728 35970 2265...
3,13748.0,22423 22086 22961 22457 84946 22077 22952 2208...
4,15100.0,21258 21259 21257 21936 22567 22627 23310 2159...


(4356, 2)
Wall time: 717 ms
